In [1]:
from bs4 import BeautifulSoup
from bs4.element import Comment
from urlparse import urlparse
import pandas as pd
import openpyxl as op
import os
import urllib2, re
import ssl
import time
import glob



os.chdir('/Users/krothaps/Desktop/hackathon/demo')

In [2]:
def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    if isinstance(element, Comment):
        return False
    return True

def fetch_text_from_url(url):
    html = urllib2.urlopen(url).read()
    soup = BeautifulSoup(html, "html.parser")
    data = soup.findAll(text=True)
 
    result = filter(visible, data)

    page = ' '.join(str(e.encode('utf-8')) for e in result)
    page = page.replace('\n', ' ').replace('\t', ' ').replace('\r', '')
    return page

In [6]:
def fetch_univ_url(line):
    line_list = line.strip().split(',')
    if line_list[2] != '#N/A' and '.edu' in line_list[2]:
        return line_list[2]
    elif 'http' in line_list[4]:
        return line_list[4]
    elif 'www' in line_list[4]:
        return 'http://'+line_list[4]
    elif 'edu' in line_list[4]:
        return 'http://www.'+line_list[4]
    else:
        return 'none'

In [21]:
fetch_univ_url('a,b,c,d,mit.edu')

'http://www.mit.edu'

In [ ]:
def print_urls(univ_url, univ_name):
    university_url = univ_url
    university_name = univ_name.lower().strip().replace(" ", "_")
    output_file_name = university_name+'.txt'
    first_level_urls = []
    second_level_urls = []
    parsed_urls = []
    random_flag = 0
    output_file = open(output_file_name, 'w')

    ssl._create_default_https_context = ssl._create_unverified_context

    html_page = urllib2.urlopen(university_url).read()
    soup = BeautifulSoup(html_page, "lxml")
    
    
    # fetch text from the main page
    data = soup.findAll(text=True)
    result = filter(visible, data)
    page = ' '.join(str(e.encode('utf-8')) for e in result)
    page = page.replace('\n', ' ').replace('\t', ' ').replace('\r', '')
    parsed_urls.append(university_url)
    
    output_file.write(university_name+'\t'+university_url+'\t'+university_url+'\t'+page+'\n')

    start_time = time.time()

    tag_data1 = soup.findAll('a', attrs={'href': re.compile("^http://")})
    tag_data2 = soup.findAll('a', attrs={'href': re.compile("^https://")})
    links1 = [link.get('href').encode('utf-8').strip() for link in tag_data1]
    links2 = [link.get('href').encode('utf-8').strip() for link in tag_data2]
    links = list(set(links1+links2))
#     print links
 
            
    execution_time = time.time() - start_time

    print univ_name+' : '+univ_url
    print "First Level URLs : "+str(len(links))
    print "Execution time : "+str(int(execution_time))+" seconds"
    
    for link in links:
        print link
    
    output_file.close()

In [24]:
print_urls('http://www.mit.edu/', 'mit')

mit : http://www.mit.edu/
First Level URLs : 16
Execution time : 0 seconds
http://whereis.mit.edu/
http://news.mit.edu/topic/synthetic-biology
http://news.mit.edu/2018/inexpensive-biology-kits-offer-hands-experience-dna-0801
https://www.google.com/maps/place/Massachusetts+Institute+of+Technology/@42.360091,-71.09416,15z/data=!4m5!3m4!1s0x0:0xd0e08ea5b308203c!8m2!3d42.360091!4d-71.09416
https://www.youtube.com/mit
http://calendar.mit.edu/
http://web.mit.edu/people.html
https://www.facebook.com/MITnews
http://careers.mit.edu/
https://twitter.com/share?url=http://web.mit.edu/spotlight/synthetic-biology-kits
https://www.instagram.com/mitpics/
http://news.mit.edu/
https://www.facebook.com/sharer/sharer.php?u=http://web.mit.edu/spotlight/synthetic-biology-kits
http://be.mit.edu/
http://connect.mit.edu/
https://twitter.com/mit


In [34]:
def fetch_recursive_url_text(univ_url, univ_name, specific_link):
    university_url = univ_url
    university_name = univ_name.lower().strip().replace(" ", "_")
    output_file_name = university_name+'.txt'
    first_level_urls = []
    second_level_urls = []
    parsed_urls = []
    random_flag = 0
    output_file = open(output_file_name, 'w')

    ssl._create_default_https_context = ssl._create_unverified_context

    html_page = urllib2.urlopen(university_url).read()
    soup = BeautifulSoup(html_page, "lxml")
    
    
    # fetch text from the main page
    data = soup.findAll(text=True)
    result = filter(visible, data)
    page = ' '.join(str(e.encode('utf-8')) for e in result)
    page = page.replace('\n', ' ').replace('\t', ' ').replace('\r', '')
    parsed_urls.append(university_url)
    
#     output_file.write(university_name+'\t'+university_url+'\t'+university_url+'\t'+page+'\n')

    start_time = time.time()

    tag_data1 = soup.findAll('a', attrs={'href': re.compile("^http://")})
    tag_data2 = soup.findAll('a', attrs={'href': re.compile("^https://")})
    links1 = [link.get('href').encode('utf-8').strip() for link in tag_data1]
    links2 = [link.get('href').encode('utf-8').strip() for link in tag_data2]
    links = list(set(links1+links2))
#     print links
 
    for link in links:
        if link != specific_link:
            continue
        try:
            if link not in parsed_urls and '.edu' in link:
                url_text = fetch_text_from_url(link)
                output_file.write(university_name+'\t'+university_url+'\t'+link+'\t'+ url_text+'\n')
            else:
                url_text = 'not-parsed'
                output_file.write(university_name+'\t'+university_url+'\t'+link+'\t'+ url_text+'\n')
            parsed_urls.append(link)    
        except:
            random_flag += 1
        print link
        print url_text
    execution_time = time.time() - start_time

#     print univ_name+' : '+univ_url
#     print "First Level URLs : "+str(len(links))
#     print "Execution time : "+str(int(execution_time))+" seconds"
#     print "***done***"
    output_file.close()

In [35]:
fetch_recursive_url_text('http://www.mit.edu/', 'mit_demo', 'http://news.mit.edu/topic/synthetic-biology')

http://news.mit.edu/topic/synthetic-biology
                                                                                         Massachusetts Institute of Technology                                News   Video   Social     Follow MIT     MIT News RSS   Follow MIT on Twitter   Follow MIT on Facebook   Follow MIT on Google+   Follow MIT on Instagram   Follow MIT on Flickr   Follow MIT on YouTube                           MIT News Office               Browse   or                         Browse       Most Popular                           Q&A: Anne McCants on "Waves of Globalization," the 2018 World Economic History Congress at MIT           China could face deadly heat waves due to climate change         Abdul Latif Jameel World Education Lab welcomes initial members         Featured video: If the spacesuit fits         Inventing future fabrics         Boeing will be Kendall Square Initiative’s first major tenant         On-chip optical filter processes wide range of light wavelength

In [7]:
with open('/Users/krothaps/Desktop/hackathon/demo/schools_list.csv','rU') as f:
    content = f.readlines()

# content = [x.strip() for x in content][0]
# print len(content)
# lines = content.split('\n')

# print len(content)
# print content
# print len(lines)

for line in content:

    if len(line.strip().split(',')) == 5:
        
        univ_name = line.strip().split(',')[0]
        output_file_name = str(univ_name)+'.txt'

        univ_url = fetch_univ_url(line)
        if not os.path.isfile(output_file_name) and 'edu' in univ_url and univ_url != 'none':
            try:
                print_urls(univ_url, line.strip().split(',')[1])
            except:
                print 'hi'
    break
    
f.close()

Western Governors University : http://www.wgu.edu
First Level URLs : 32
Execution time : 0 seconds
https://www.wgu.edu
https://northcarolina.wgu.edu
https://www.wgu.edu/landing/responsibleborrowing
https://texas.wgu.edu
https://wwwforms.wgu.edu/wgu/app/app_step0
https://tennessee.wgu.edu
https://www.wgu.edu/tuition_financial_aid/scholarships/backtoschool?CID=HOMEpromo_Back-School-Scholarship-2018-btn
https://www.wgu.edu/admissions.html
https://plus.google.com/+wgu
https://www.instagram.com/westerngovernorsu/
https://nevada.wgu.edu
https://twitter.com/wgu
https://www.wgu.edu/online-it-degrees/bachelors-programs.html
https://www.wgu.edu/online-business-degrees/masters-programs.html
https://washington.wgu.edu
https://missouri.wgu.edu
https://www.wgu.edu/online-business-degrees/bachelors-programs.html
https://www.youtube.com/user/WesternGovernorsUniv
https://indiana.wgu.edu
https://www.wgu.edu/online-it-degrees/masters-programs.html
https://www.wgu.edu/online-teaching-degrees/education-mas

###### Fetch the cotent from the URLs

In [31]:
# with open('/Users/krothaps/Desktop/hackathon/demo/schools_list.csv','rU') as f:
#     content = f.readlines()

# # you may also want to remove whitespace characters like `\n` at the end of each line
# # content = [x.strip() for x in content][0]
# # print len(content)
# # lines = content.split('\n')

# # print len(content)
# # print content
# # print len(lines)

# for line in content:

#     if len(line.strip().split(',')) == 5:
        
#         univ_name = line.strip().split(',')[0]
#         output_file_name = str(univ_name)+'.txt'

#         univ_url = fetch_univ_url(line)
#         if not os.path.isfile(output_file_name) and 'edu' in univ_url and univ_url != 'none':
#             try:
#                 fetch_recursive_urls(univ_url, line.strip().split(',')[1])
#             except:
#                 print 'hi'
#     break
    
# f.close()

hi


In [32]:
# Extract data from twitter

def fetch_twitter_handle():
    for filename in glob.glob(os.path.join('*.txt')):
        with open(filename) as f:
            content = f.readlines()
        
    for line in content:
        line = line.decode('utf-8')
        first_level_url = line.split('\t')[2].strip()
        if "twitter.com/" in first_level_url:
            print first_level_url
        
fetch_twitter_handle()
        

In [33]:
# Extract all the twitter handles

def fetch_twitter_handles():
    twitter_handles = []
    url_count = 0
    exception_count = 0
    for filename in glob.glob(os.path.join('./fetched_data/*.txt')):
        with open(filename) as f:
            content = f.readlines()
        f.close()
        for line in content:
            url_count += 1
            line = line.decode('utf-8')
            try:
                first_level_url = line.split(',')[1].strip()
                if "http://twitter.com/" in first_level_url and first_level_url.count('/') == 3 and '%' not in first_level_url and '?' not in first_level_url:
                    twitter_handles.append(first_level_url)
            except:
                exception_count += 1
                
                
    print "Parsed "+str(url_count)+" URLs"
    twitter_handles = list(set(twitter_handles))
    print "Found "+str(len(twitter_handles))+" twitter handles"
    for handle in twitter_handles:
        print handle
    
fetch_twitter_handles()
        

Parsed 1473201 URLs
Found 971 twitter handles
http://twitter.com/MSU_WBasketball
http://twitter.com/Albion_Softball
http://twitter.com/csufsli
http://twitter.com/ucla
http://twitter.com/bowdoincollege
http://twitter.com/IllinoisSARE
http://twitter.com/loyola_NOLA
http://twitter.com/AlmaScotsAD
http://twitter.com/depaulnewsroom
http://twitter.com/nwciowa
http://twitter.com/conncollege
http://twitter.com/MayorLachapelle
http://twitter.com/CSUF
http://twitter.com/TXChiroCollege
http://twitter.com/sdsuwireless
http://twitter.com/ucralum
http://twitter.com/MSUswimdive
http://twitter.com/neurowitz
http://twitter.com/uie_money
http://twitter.com/iaslibrary
http://twitter.com/dugoni_dental
http://twitter.com/LondonBreed
http://twitter.com/UCSDtritons
http://twitter.com/Illinois4H
http://twitter.com/msutrackfieldxc
http://twitter.com/AlbionRiding
http://twitter.com/stmarysca
http://twitter.com/CreatingValue
http://twitter.com/drakelibguy
http://twitter.com/uhartfordnews
http://twitter.com/miami

http://twitter.com/LibFSU
http://twitter.com/asu
http://twitter.com/ksuignition
http://twitter.com/loyolamaryland
http://twitter.com/fiumedicine
http://twitter.com/BabsonSFS
http://twitter.com/UofIFS
http://twitter.com/msuwomensgolf
http://twitter.com/MaryvilleC
http://twitter.com/regiscollege_ma
http://twitter.com/leeu
http://twitter.com/PrincipiaSchool
http://twitter.com/NASA
http://twitter.com/AzusaPacific
http://twitter.com/NatAmericanU
http://twitter.com/CentennialCCU
http://twitter.com/illinois_alma
http://twitter.com/NDU_iCollege
http://twitter.com/sdsucareerserv
http://twitter.com/uninews
http://twitter.com/GCconnect
http://twitter.com/cospar2018
http://twitter.com/ucdavisvetmed
http://twitter.com/OhioDominican
http://twitter.com/@Sustainable_EKU
http://twitter.com/bryantmuseum
http://twitter.com/ncatsuaggies
http://twitter.com/WrestleMcDaniel
http://twitter.com/uvmvermont
http://twitter.com/TerpsFootball
http://twitter.com/MSUfieldhockey
http://twitter.com/frostburgstate
http:

In [11]:
#Configure twitter API settings

from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys
import os
import time

#Variables that contains the user credentials to access Twitter API 
access_token = "23009567-uhZDJWZwQ8CO7SBHN5lqDg2ft6ftmJ2RdKFV5H2E9"
access_token_secret = "8Yavfj4v8zWhgcRxOWRlE2UbLq6RqU6esWX8Mk5i0M"

consumer_key = "D9S2hZKTwg1vkRcaFT9djA"
consumer_secret = "rpe3pctrZWeFa0YWyjvnZMeLi47fwZ0DHk23TjmSPU"
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth)

In [12]:
# Extract metadatra of a handle and its tweets 

account_list = ['wgu']
random_flag = 1
if len(account_list) > 0:
  for target in account_list:
    print("Getting data for " + target)
    try:
        item = auth_api.get_user(target)
        print "name: " + item.name
        print "screen_name: " + item.screen_name
        print "description: " + item.description
        print "statuses_count: " + str(item.statuses_count)
        print "friends_count: " + str(item.friends_count)
        print "followers_count: " + str(item.followers_count)
#         output_line = item.name.encode('utf-8')+'\t'+ item.screen_name.encode('utf-8')+'\t'+ item.description.encode('utf-8')+'\t'+ str(item.statuses_count)+'\t'+str(item.friends_count)+'\t'+str(item.followers_count)+'\t'+item.location.encode('utf-8')+'\t'+str(item.created_at)+'\n'
    except:
        random_flag += 1



Getting data for wgu
name: Western Governors U.
screen_name: wgu
description: Online, accredited, competency-based university. Flexibility + superior results at half the cost. #HigherEd #CBE #NonProfit Also follow our president, @scottpul
statuses_count: 7891
friends_count: 11785
followers_count: 20598


In [13]:
hashtags = []
mentions = []
tweet_count = 0
target = 'wgu'
end_date = datetime.utcnow() - timedelta(days=30)
for status in Cursor(auth_api.user_timeline, id=target).items():
	tweet_count += 1
# 	print status
	if hasattr(status, "entities"):
		entities = status.entities
		if "hashtags" in entities:
			for ent in entities["hashtags"]:
				if ent is not None:
					if "text" in ent:
						hashtag = ent["text"]
						if hashtag is not None:
							hashtags.append(hashtag)
		if "user_mentions" in entities:
			for ent in entities["user_mentions"]:
				if ent is not None:
					if "screen_name" in ent:
						name = ent["screen_name"]
						if name is not None:
							mentions.append(name)
# 		if status.created_at < end_date:
# 			break

In [14]:
print status

Status(contributors=None, truncated=False, text=u'@jlongoria21 @scottpul We\u2019ll share that feedback with mentoring leadership so they can be sure their team keeps all resources in mind.', is_quote_status=False, in_reply_to_status_id=731224934656024576, id=731225135869394944, favorite_count=0, _api=<tweepy.api.API object at 0x1129704d0>, author=User(follow_request_sent=False, has_extended_profile=True, profile_use_background_image=True, _json={u'follow_request_sent': False, u'has_extended_profile': True, u'profile_use_background_image': True, u'default_profile_image': False, u'id': 15291876, u'profile_background_image_url_https': u'https://abs.twimg.com/images/themes/theme1/bg.png', u'verified': True, u'translator_type': u'none', u'profile_text_color': u'333333', u'profile_image_url_https': u'https://pbs.twimg.com/profile_images/773256478576644096/TYpNrg_I_normal.jpg', u'profile_sidebar_fill_color': u'F3F3F3', u'entities': {u'url': {u'urls': [{u'url': u'http://t.co/jjsWYKuBWw', u'in

In [15]:
print
print("Most mentioned Twitter users:")
for item, count in Counter(mentions).most_common(10):
    print(item + "\t" + str(count))
 
print
print("Most used hashtags:")
for item, count in Counter(hashtags).most_common(10):
    print(item + "\t" + str(count))
 
print
print "All done. Processed " + str(tweet_count) + " tweets."
print


Most mentioned Twitter users:
wgu	542
scottpul	164
Owls2Work	52
WGUTexas	47
WGUWashington	39
PrincipalKafele	37
WGUIndiana	32
WGUTennessee	32
jjonesschenk	27
sekouandrews	23

Most used hashtags:
WGU	328
WGUgrad	222
WGUIT	130
WGUSageTalks	117
InnovationWins	76
WGUforLife	58
WGUnursing	57
IT	52
CBE	48
WGU20th	39

All done. Processed 3224 tweets.



In [16]:
# Save twitter data

filename = target+'_twitter_data1.txt'
tweets_metadata_file = open(filename, 'w')
random_flag = 1

for target in account_list:
    while True:
        try:
            for status in Cursor(auth_api.user_timeline, id=target).items():
                tweets_metadata_file.write(target + '\t' + status.text.replace('\n',' ').replace('\t',' ').encode('utf-8') + '\t'+ str(status.created_at) + '\t'+ str(status.retweet_count)+ '\t'+str(status.favorite_count) + '\t'+ str(status.in_reply_to_user_id)+ '\n')
        except tweepy.TweepError:
            time.sleep(60 * 15)
            continue
        except StopIteration:
            pass
        break
    
        
tweets_metadata_file.close()

In [17]:
# Data fetched from twitter

num_lines1 = sum(1 for line in open('/Users/krothaps/Desktop/hackathon/dataset/twitter-stream/twitter_stream.txt'))
num_lines2 = sum(1 for line in open('/Users/krothaps/Desktop/hackathon/dataset/twitter-stream/twitter_stream1.txt'))
num_lines3 = sum(1 for line in open('/Users/krothaps/Desktop/hackathon/dataset/twitter-stream/twitter_stream2.txt'))

print "Fetched Tweets : "+str(num_lines1+num_lines2+num_lines3)

with open('tweets_sorted.txt','rU') as f:
    tweets = f.readlines()


print tweets[0]


Fetched Tweets : 1891298
73385 RT @amouurr_a: Seniors in highschool: idk what I want to do yet, or how will I pay for college w/o getting loans  Army Recruiters: https://…



In [18]:
import random

print(random.choice(tweets))


   1 @turtlesowateva isn't college starting soon? Hope that will keep your mind off of things a little. Just know that I am here for you. 💕❤

